# Agent 4: Data Analyzer
**`Data Analyzer`** is an agent focused on data analysis, responsible for extracting valuable information from structured data (such as CSV files exported from databases), supporting two main tasks:

1. **Anomaly Detection**: Identifying potential anomalous events in network security (such as abnormal traffic, unauthorized access, or attack behaviors).
2. **Trend Prediction**: Predicting future security situations based on historical data (such as attack frequency trends, network traffic changes, and potential threat activity).

Working in conjunction with **TimeGPT** (time series analysis tool), **Data Analyzer** can generate intuitive analysis results and return them in visual or data format to assist with network security decision-making.

In [5]:
# load environment variables from the .env file
from dotenv import load_dotenv

load_dotenv()

True

Define - analyze_data

分析數據集並執行指定的分析方法（預測或異常檢測）
此函數根據 file_name 參數讀取預定義的數據集，執行選定的分析方法（forecast 或 anomaly_detection），並以 JSON 格式 返回結果。

若選擇 預測（forecast），則根據歷史數據生成未來的預測值，可設定 預測範圍（horizon），並可選擇進行 微調步驟（fine-tuning steps）。

若選擇 異常檢測（anomaly_detection），則識別數據中的異常點。

函數參數（Parameters）
file_name（str）：要分析的數據集名稱。

method（str）：選擇的分析方法，必須為 forecast 或 anomaly_detection：

forecast：基於歷史數據預測未來值。

anomaly_detection：識別數據中的異常模式。

horizon（int，選填）：預測範圍，即要預測的未來步數。僅適用於 forecast 方法，預設為 0。

ftSteps（int，選填）：執行 微調步驟 的次數。僅適用於 forecast 方法，預設為 0。

函數返回（Return）
以 JSON 字符串 返回分析結果，具體內容依 method 選擇的分析方法而定：

對於 forecast，結果包括 預測值 和 可視化圖表。

對於 anomaly_detection，結果包含 檢測出的異常數據點 和 可視化圖表。

若發生錯誤，則返回包含 錯誤訊息 的 JSON 結構。

返回類型（rtype）：str（JSON 格式）

In [17]:
import os
import json
import pandas as pd
from typing import Any, Optional, Literal, Set, Callable
from nixtla import NixtlaClient


def analyze_data(file_path: str,
                 method: Literal['forecast', 'anomaly_detection'],
                 horizon: Optional[int] = 0,
                 ftSteps: Optional[int] = 0) -> str:
    """
    Analyze a dataset using the specified analysis method (forecasting or anomaly detection).

    This function reads a pre-defined dataset based on the provided `file_name` argument, 
    performs the selected analysis method (`forecast` or `anomaly_detection`), and 
    returns the results in JSON format. If forecasting is selected, it generates 
    predictions over a specified horizon and optionally uses fine-tuning steps. 
    If anomaly detection is selected, it identifies anomalies in the data.

    :param file_name: The name of the dataset to analyze. 

    :param method: The analysis method to use. Must be one of ['forecast', 'anomaly_detection']:
                   - 'forecast': Predicts future values based on historical data.
                   - 'anomaly_detection': Detects anomalies in the data.

    :param horizon: (Optional) The forecast horizon, i.e., the number of future steps to predict.
                    Only applicable when `method='forecast'`. Default is 0.

    :param ftSteps: (Optional) The number of fine-tuning steps to apply during forecasting.
                    Only applicable when `method='forecast'`. Default is 0.

    :return: A JSON string containing the results of the analysis. The structure of the output depends on
             the selected method:
             - For `forecast`, it includes forecasted values and a visualization.
             - For `anomaly_detection`, it includes detected anomalies and a visualization.
             If an error occurs, the returned JSON contains an error message.
    :rtype: str
    """
    
    if os.getenv("TIME_GEN_ENDPOINT") is None or os.getenv("TIME_GEN_KEY") is None:
        if method == 'forecast':
            return "./figures/Intrusion_Attempts_forecast_plot.png"
        else:
            return "./figures/Incident_Detection_Rate_anomalies_plot.png"
    
    nixtla_client = NixtlaClient(
        base_url=os.getenv("TIME_GEN_ENDPOINT"),
        api_key=os.getenv("TIME_GEN_KEY"),
    )

    if file_path is not None:
        file_path = file_path if file_path.endswith(
            'csv') else f'./data/{file_path}.csv'
        file_name = file_path.split('/')[-1].split('.')[0]
        df = pd.read_csv(file_path)

        print(df.head())  # 查看前幾行數據
        print(df.info())  # 查看數據框的結構
        df['timestamp'] = pd.to_datetime(df['timestamp'])
        print(df.info())  # 查看數據框的結構
        
        # Initialize variables
        horizon_int = None
        ftSteps_int = None

        try:
            horizon_int = int(horizon)
        except (ValueError, TypeError) as e:
            return f"Error converting 'horizon': {e}"

        try:
            ftSteps_int = int(ftSteps)
        except (ValueError, TypeError) as e:
            return f"Error converting 'ftSteps': {e}"

        if ftSteps is None:
            ftSteps_int = 0

        try:
            if method == 'forecast':
                try:
                    forecast_df = nixtla_client.forecast(
                        df=df,
                        h=horizon_int,
                        finetune_steps=ftSteps_int,
                        time_col="timestamp",
                        target_col="value",
                    )
                except Exception as e:
                    print(f"Error during forecast: {e}")
                    return json.dumps({"error": str(e)})
                fig = nixtla_client.plot(
                    df=df, forecasts_df=forecast_df, time_col="timestamp", target_col="value"
                )

                ax = fig.axes[0]
                ax.legend(["Actual Values", "Forecasted Values"],
                          loc="upper right", bbox_to_anchor=(1.14, 1), borderaxespad=0)
                ax.set_title(f"{file_name}: Forecasted vs Actual Values", fontsize=18,
                             fontweight="bold", color="teal", pad=15)

                fig_name = f"./figures/{file_name}_forecast_plot.png"
                fig.savefig(fig_name, dpi=300)

                return fig_name
            elif method == "anomaly_detection":
                anomalies_df = nixtla_client.detect_anomalies(
                    df,
                    time_col="timestamp",
                    target_col="value",
                    freq="D",
                )
                anomalies_df = anomalies_df.rename(columns={
                    "TimeGEN": "TimeGPT",
                    "TimeGEN-lo-99": "TimeGPT-lo-99",
                    "TimeGEN-hi-99": "TimeGPT-hi-99"
                })

                fig = nixtla_client.plot(
                    df, anomalies_df, time_col="timestamp", target_col="value")

                ax = fig.axes[0]
                ax.legend(["Actual Values", "TimeGPT", "TimeGPT_level_99", "TimeGPT_anomalies_level_99"],
                          loc="upper right", bbox_to_anchor=(1.21, 1), borderaxespad=0)
                ax.set_title(f"{file_name}: Anomalies on Actual Values", fontsize=18,
                             fontweight="bold", color="teal", pad=15)

                fig_name = f"./figures/{file_name}_anomalies_plot.png"
                fig.savefig(fig_name, dpi=300)

                return fig_name
            else:
                return "Invalid method specified."
        except Exception as e:
            return json.dumps({"error": str(e)})
    else:
        return "No data file provided."

In [18]:
analyze_data("./data/intrusion_attempts.csv", 'forecast', horizon=10)

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS


    timestamp  value
0  1949-01-01    112
1  1949-02-01    118
2  1949-03-01    132
3  1949-04-01    129
4  1949-05-01    121
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   timestamp  144 non-null    object
 1   value      144 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 2.4+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 144 entries, 0 to 143
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  144 non-null    datetime64[ns]
 1   value      144 non-null    int64         
dtypes: datetime64[ns](1), int64(1)
memory usage: 2.4 KB
None


INFO:nixtla.nixtla_client:Attempt 1 failed...
INFO:nixtla.nixtla_client:Attempt 2 failed...
INFO:nixtla.nixtla_client:Attempt 3 failed...
INFO:nixtla.nixtla_client:Attempt 4 failed...
INFO:nixtla.nixtla_client:Attempt 5 failed...
INFO:nixtla.nixtla_client:Attempt 6 failed...


Error during forecast: 1 validation error for ParsingModel[HttpValidationError]
__root__ -> detail
  value is not a valid list (type=type_error.list)


'{"error": "1 validation error for ParsingModel[HttpValidationError]\\n__root__ -> detail\\n  value is not a valid list (type=type_error.list)"}'

In [4]:
analyze_data("./data/incident_detection_rate.csv", 'anomaly_detection')

INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Anomaly Detector Endpoint...
INFO:nixtla.nixtla_client:Attempt 1 failed...
INFO:nixtla.nixtla_client:Attempt 2 failed...
INFO:nixtla.nixtla_client:Attempt 3 failed...
INFO:nixtla.nixtla_client:Attempt 4 failed...
INFO:nixtla.nixtla_client:Attempt 5 failed...
INFO:nixtla.nixtla_client:Attempt 6 failed...


'{"error": "1 validation error for ParsingModel[HttpValidationError]\\n__root__ -> detail\\n  value is not a valid list (type=type_error.list)"}'

## Create Agent

In [9]:
import os
import logging
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.projects.models import FunctionTool


project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["AIPROJECT_CONNECTION_STRING"]
)

logging.getLogger('azure.core.pipeline.policies.http_logging_policy').setLevel(logging.WARNING)

INFO:azure.identity._credentials.environment:No environment configuration found.
INFO:azure.identity._credentials.managed_identity:ManagedIdentityCredential will use IMDS


In [10]:
from typing import Any, Set, Callable
from azure.ai.projects.models import FunctionTool


user_functions: Set[Callable[..., Any]] = {analyze_data}
functions = FunctionTool(functions=user_functions)
print(functions.definitions)

[{'type': 'function', 'function': {'name': 'analyze_data', 'description': 'Analyze a dataset using the specified analysis method (forecasting or anomaly detection).', 'parameters': {'type': 'object', 'properties': {'file_path': {'type': 'string', 'description': 'No description'}, 'method': {'type': 'string', 'description': "The analysis method to use. Must be one of ['forecast', 'anomaly_detection']:"}, 'horizon': {'type': ['integer', 'null'], 'description': '(Optional) The forecast horizon, i.e., the number of future steps to predict.'}, 'ftSteps': {'type': ['integer', 'null'], 'description': '(Optional) The number of fine-tuning steps to apply during forecasting.'}}, 'required': ['file_path', 'method']}}}]


In [11]:
data_analyzer = project_client.agents.create_agent(
    model=os.environ["CHAT_MODEL"],
    name="data_analyzer",
    description="An agent that analyzes data using forecasting or anomaly detection.",
    instructions="Hello, you are helpful assistant.",
    tools=functions.definitions,
    # Parameters
    temperature=0.7,
    top_p=0.95,
    # Metadata
    metadata={"group": "internet_threat_analysis"},
)

print(f"Created agent, agent ID: {data_analyzer.id}")

INFO:azure.identity._credentials.chained:DefaultAzureCredential acquired a token from AzureCliCredential


Created agent, agent ID: asst_TZ8XzqpP0rld787V80Nv3Yhb


In [12]:
import time
from azure.ai.projects.models import RequiredFunctionToolCall, SubmitToolOutputsAction, ToolOutput


agent_id = data_analyzer.id


thread = project_client.agents.create_thread()
print(f"Created thread, ID: {thread.id}")

# Create message to thread
message_1 = project_client.agents.create_message(
    thread_id=thread.id, role="assistant", content="""The intrusion attempts data has been successfully fetched and saved as a CSV file. You can find it at the following path: ./data/intrusion_attempts.csv.
    
    The incident detection data has been successfully fetched and saved as a CSV file. You can find it at the following path: ./data/incident_detection_rate.csv
    """
)
message_2 = project_client.agents.create_message(
    thread_id=thread.id, role="user", content="Hello, create a 10-day forecast using the intrusion attempts data. And detect anomalies in the incident detection data."
)

# Create and process assistant run in thread with tools
run = project_client.agents.create_run(thread_id=thread.id, assistant_id=agent_id)
print(f"Created run, ID: {run.id}")

while run.status in ["queued", "in_progress", "requires_action"]:
    time.sleep(1)
    run = project_client.agents.get_run(thread_id=thread.id, run_id=run.id)

    if run.status == "requires_action" and isinstance(run.required_action, SubmitToolOutputsAction):
        tool_calls = run.required_action.submit_tool_outputs.tool_calls
        if not tool_calls:
            print("No tool calls provided - cancelling run")
            project_client.agents.cancel_run(
                thread_id=thread.id, run_id=run.id)
            break

        tool_outputs = []
        for tool_call in tool_calls:
            if isinstance(tool_call, RequiredFunctionToolCall):
                try:
                    print(f"Executing tool call: {tool_call}")
                    output = functions.execute(tool_call)
                    tool_outputs.append(
                        ToolOutput(
                            tool_call_id=tool_call.id,
                            output=output,
                        )
                    )
                except Exception as e:
                    print(f"Error executing tool_call {tool_call.id}: {e}")

        print(f"Tool outputs: {tool_outputs}")
        if tool_outputs:
            project_client.agents.submit_tool_outputs_to_run(
                thread_id=thread.id, run_id=run.id, tool_outputs=tool_outputs
            )

    print(f"Current run status: {run.status}")

print(f"Run completed with status: {run.status}")

Created thread, ID: thread_a9Nl41ZKfML2YBN5cNODwAGw
Created run, ID: run_GFqEnGcdm2AJ1FVqb71DU2Vh
Current run status: RunStatus.IN_PROGRESS
Executing tool call: {'id': 'call_MCHNAmS2q7v1RGKjgsLhiY5v', 'type': 'function', 'function': {'name': 'analyze_data', 'arguments': '{"file_path": "./data/intrusion_attempts.csv", "method": "forecast", "horizon": 10}'}}


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Attempt 1 failed...
INFO:nixtla.nixtla_client:Attempt 2 failed...
INFO:nixtla.nixtla_client:Attempt 3 failed...
INFO:nixtla.nixtla_client:Attempt 4 failed...
INFO:nixtla.nixtla_client:Attempt 5 failed...
INFO:nixtla.nixtla_client:Attempt 6 failed...


Executing tool call: {'id': 'call_0egTsM1WmbGIC1eFl771hK5G', 'type': 'function', 'function': {'name': 'analyze_data', 'arguments': '{"file_path": "./data/incident_detection_rate.csv", "method": "anomaly_detection"}'}}


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Anomaly Detector Endpoint...
INFO:nixtla.nixtla_client:Attempt 1 failed...
INFO:nixtla.nixtla_client:Attempt 2 failed...
INFO:nixtla.nixtla_client:Attempt 3 failed...
INFO:nixtla.nixtla_client:Attempt 4 failed...
INFO:nixtla.nixtla_client:Attempt 5 failed...
INFO:nixtla.nixtla_client:Attempt 6 failed...


Tool outputs: [{'tool_call_id': 'call_MCHNAmS2q7v1RGKjgsLhiY5v', 'output': '{"error": "1 validation error for ParsingModel[HttpValidationError]\\n__root__ -> detail\\n  value is not a valid list (type=type_error.list)"}'}, {'tool_call_id': 'call_0egTsM1WmbGIC1eFl771hK5G', 'output': '{"error": "1 validation error for ParsingModel[HttpValidationError]\\n__root__ -> detail\\n  value is not a valid list (type=type_error.list)"}'}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Current run status: RunStatus.IN_PROGRESS
Executing tool call: {'id': 'call_Fj70nzlQCegDrAT4M6Rq78p9', 'type': 'function', 'function': {'name': 'analyze_data', 'arguments': '{"file_path": "./data/intrusion_attempts.csv", "method": "forecast", "horizon": 10}'}}


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Inferred freq: MS
INFO:nixtla.nixtla_client:Attempt 1 failed...
INFO:nixtla.nixtla_client:Attempt 2 failed...
INFO:nixtla.nixtla_client:Attempt 3 failed...
INFO:nixtla.nixtla_client:Attempt 4 failed...
INFO:nixtla.nixtla_client:Attempt 5 failed...
INFO:nixtla.nixtla_client:Attempt 6 failed...


Executing tool call: {'id': 'call_tVDc6AFQTmYEs61cWAdXjeGH', 'type': 'function', 'function': {'name': 'analyze_data', 'arguments': '{"file_path": "./data/incident_detection_rate.csv", "method": "anomaly_detection"}'}}


INFO:nixtla.nixtla_client:Validating inputs...
INFO:nixtla.nixtla_client:Preprocessing dataframes...
INFO:nixtla.nixtla_client:Calling Anomaly Detector Endpoint...
INFO:nixtla.nixtla_client:Attempt 1 failed...
INFO:nixtla.nixtla_client:Attempt 2 failed...
INFO:nixtla.nixtla_client:Attempt 3 failed...
INFO:nixtla.nixtla_client:Attempt 4 failed...
INFO:nixtla.nixtla_client:Attempt 5 failed...
INFO:nixtla.nixtla_client:Attempt 6 failed...


Tool outputs: [{'tool_call_id': 'call_Fj70nzlQCegDrAT4M6Rq78p9', 'output': '{"error": "1 validation error for ParsingModel[HttpValidationError]\\n__root__ -> detail\\n  value is not a valid list (type=type_error.list)"}'}, {'tool_call_id': 'call_tVDc6AFQTmYEs61cWAdXjeGH', 'output': '{"error": "1 validation error for ParsingModel[HttpValidationError]\\n__root__ -> detail\\n  value is not a valid list (type=type_error.list)"}'}]
Current run status: RunStatus.REQUIRES_ACTION
Current run status: RunStatus.COMPLETED
Run completed with status: RunStatus.COMPLETED


In [13]:
from IPython.display import Markdown, display
import helper

messages = project_client.agents.list_messages(thread_id=thread.id)

display(Markdown(helper.get_conversation_md(messages)))

INFO:azure.identity._internal.decorators:AzureCliCredential.get_token_info succeeded
INFO:azure.identity._credentials.default:DefaultAzureCredential acquired a token from AzureCliCredential


# Conversation
___
### **Assistant** (2025-03-27 19:13:07 台北標準時間)
The intrusion attempts data has been successfully fetched and saved as a CSV file. You can find it at the following path: ./data/intrusion_attempts.csv.

    The incident detection data has been successfully fetched and saved as a CSV file. You can find it at the following path: ./data/incident_detection_rate.csv
    
___
### **User** (2025-03-27 19:13:08 台北標準時間)
Hello, create a 10-day forecast using the intrusion attempts data. And detect anomalies in the incident detection data.
___
### **Assistant** (2025-03-27 19:15:13 台北標準時間)
It seems there was an error while attempting to process the data for forecasting and anomaly detection. Let me try again to perform the analyses.
___
### **Assistant** (2025-03-27 19:17:13 台北標準時間)
It appears that there is a persistent issue with processing the data files for both the forecasting and anomaly detection. The errors indicate that the input values are not valid lists, suggesting a potential issue with the data format or content in the CSV files.

To resolve this, I can check the contents of the CSV files to ensure they are properly formatted. Would you like me to do that?
___

In [ ]:
messages